## 각 차량의 방향(전면, 후면, 옆면)에 해당하는 사진을 추출 및 타겟 위치로 복사
- 데이터셋 제작 단계에 해당하는 부분입니다.
- 차량의 경우, 사진의 각도에 따라 그 모습이 상이하기에 차량에 방향에 따라 각각 모델링을 진행하게 됩니다. 그러므로 데이터도 각각 따로 만들어줍니다. 
- 차량 특정 파트의 존재 유무를 통해 차량이 향하는 방향을 알 수 있으며, 파트들의 상대적 크기 비율을 통해 차량의 각도를 보다 자세히 유추할 수 있습니다.
- 차량의 전체가 찍힌 사진만 사용합니다. 그리고 차량 객체의 위치와 크기에 맞게 크랍해줍니다.
- 데이터셋 생성 이후, 수작업으로 연식별로 분류된 차량들을 연형별로 분류해줍니다.  

In [ ]:
import os
import json
from PIL import Image
import shutil

def get_crop_coordinates(json_data, label_name):
    """지정된 레이블(P00.차량전체)을 기준으로 크롭 좌표 추출"""
    for shape in json_data['shapes']:
        if shape['label'] == label_name:
            points = shape['points']
            x_coords = [p[0] for p in points]
            y_coords = [p[1] for p in points]
            return min(x_coords), min(y_coords), max(x_coords), max(y_coords)
    return None


def validate_conditions(json_data, folder_type):
    """각 차량 방향에 적용되는 바운딩박스 조건 함수 (수정된 version)"""
    labels = {shape['label'] for shape in json_data['shapes']}
    if folder_type == 'car_back':
        # "car_back"에 대한 조건
        # 차량전체 + 헤드램프 없음
        if "P00.차량전체" in labels and "P10.헤드램프" not in labels:
            rear_lamps = [shape for shape in json_data['shapes'] if shape['label'] == "P11.리어램프"]
            # 리어램프가 하나일 경우 bounding box 비율이 1:6을 넘는 사진
            # 추후 1:4로 조정/ 이유: 1:6의 경우 각도가 너무 타이트해서 해당하는 사진의 수가 매우 작음
            # But 1:4로 할 경우, 리어램프가 애초에 하나인 차량이 아니라 각도에 의해 한쪽만 찍힌 차량이 해당될 경우가 크기 때문에 이에 대해 고려하여 조건 추가해야 함
            if len(rear_lamps) == 1:
                x1, y1, x2, y2 = get_crop_coordinates(json_data, "P11.리어램프")
                width = x2 - x1
                height = y2 - y1
                return width >= 6 * height
            # 리어램프가 두 개일 경우, 한 개의 리어램프의 width가 다른 하나의 리어램프의 width의 5배를 넘지 않는다 
            elif len(rear_lamps) == 2:
                widths = [get_crop_coordinates(json_data, lamp['label'])[2] - get_crop_coordinates(json_data, lamp['label'])[0] for lamp in rear_lamps]
                return not (widths[0] > 5 * widths[1] or widths[1] > 5 * widths[0])
        return False
    elif folder_type == 'car_front':
        # "car_front"에 대한 조건
        # 차량전체 + 리어램프 없음
        if "P00.차량전체" in labels and "P11.리어램프" not in labels:
            head_lamps = [shape for shape in json_data['shapes'] if shape['label'] == "P10.헤드램프"]
            # 헤드램프가 두 개일 경우, 한 개의 리어램프의 width가 다른 하나의 리어램프의 width의 5배를 넘지 않는다 
            if len(head_lamps) == 2:
                widths = [get_crop_coordinates(json_data, lamp['label'])[2] - get_crop_coordinates(json_data, lamp['label'])[0] for lamp in head_lamps]
                return not (widths[0] > 5 * widths[1] or widths[1] > 5 * widths[0])
        return False
    elif folder_type == 'car_side':
        # "car_side"에 대한 조건
        required_labels = {"P10.헤드램프", "P11.리어램프", "P00.차량전체"}
        counts = {label: sum(1 for shape in json_data['shapes'] if shape['label'] == label) for label in required_labels}
        # 각 필수 레이블이 정확히 하나만 있음
        # 헤드램프의 width가 리어램프의 width의 4배를 넘지 않는다 (vice versa)
        if all(count == 1 for count in counts.values()):
            head_lamp_width = get_crop_coordinates(json_data, "P10.헤드램프")[2] - get_crop_coordinates(json_data, "P10.헤드램프")[0]
            rear_lamp_width = get_crop_coordinates(json_data, "P11.리어램프")[2] - get_crop_coordinates(json_data, "P11.리어램프")[0]
            if not (head_lamp_width > 4 * rear_lamp_width or rear_lamp_width > 4 * head_lamp_width):
                return True
        return False
    return True


def process_directory(source_dir, target_dir, folder_type):
    """특정 규칙에 따라 각 디렉토리를 처리하고 유효한 이미지를 복사합니다."""
    for root, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.jpg'):
                json_path = os.path.join(root, file.replace('.jpg', '.json'))
                if os.path.exists(json_path):
                    with open(json_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                        if validate_conditions(data, folder_type):
                            img_path = os.path.join(root, file)
                            img = Image.open(img_path)
                            # "P00.차량전체"를 기준으로 크롭 적용
                            if "P00.차량전체" in {shape['label'] for shape in data['shapes']}:
                                x1, y1, x2, y2 = get_crop_coordinates(data, "P00.차량전체")
                                img = img.crop((x1, y1, x2, y2))
                            # 새 위치에 크롭된 이미지 저장
                            new_root = root.replace(source_dir, target_dir)
                            os.makedirs(new_root, exist_ok=True)
                            target_img_path = os.path.join(new_root, file)
                            img.save(target_img_path)

In [10]:
# 경로 정의 및 처리
# 추후 보다 효율적이고 균형적인 훈련/검증 데이터 분리를 위해 훈련/검증 데이터를 통합해 줍니다.   
base_paths = ["angle_HK_train", "angle_HK_valid"]
for base_path in base_paths: 
    print(base_path)
    target_base = "angle_jpg4"
    for folder in ['car_back', 'car_front', 'car_side']:
        source_path = os.path.join(base_path, folder)
        target_path = os.path.join(target_base, folder)
        process_directory(source_path, target_path, folder)
        print(f"all files from {source_path} processed and copied to {target_path}")

angle_HK_train
all files from angle_HK_train\car_back processed and copied to angle_jpg4\car_back
all files from angle_HK_train\car_front processed and copied to angle_jpg4\car_front
all files from angle_HK_train\car_side processed and copied to angle_jpg4\car_side
angle_HK_valid
all files from angle_HK_valid\car_back processed and copied to angle_jpg4\car_back
all files from angle_HK_valid\car_front processed and copied to angle_jpg4\car_front
all files from angle_HK_valid\car_side processed and copied to angle_jpg4\car_side


In [ ]:
# 기존 바운딩박스 조건 함수
""" def validate_conditions(json_data, folder_type):
    labels = {shape['label'] for shape in json_data['shapes']}
    if folder_type == 'car_back':
        # Conditions for "car_back"
        if "P00.차량전체" in labels and "P10.헤드램프" not in labels:
            rear_lamps = [shape for shape in json_data['shapes'] if shape['label'] == "P11.리어램프"]
            if len(rear_lamps) == 1:
                x1, y1, x2, y2 = get_crop_coordinates(json_data, "P11.리어램프")
                width = x2 - x1
                height = y2 - y1
                return width >= 6 * height
            elif len(rear_lamps) == 2:
                return True
        return False
    elif folder_type == 'car_front':
        # Conditions for "car_front"
        if "P00.차량전체" in labels and "P11.리어램프" not in labels:
            head_lamps = [shape for shape in json_data['shapes'] if shape['label'] == "P10.헤드램프"]
            if len(head_lamps) == 2:
                return True
        return False
    elif folder_type == 'car_side':
        # Conditions for "car_side" 
        required_labels = {"P10.헤드램프", "P11.리어램프", "P00.차량전체"}
        counts = {label: sum(1 for shape in json_data['shapes'] if shape['label'] == label) for label in required_labels}
        # Check for exactly one of each required label
        if all(count == 1 for count in counts.values()):
            return True
        return False
    return True """